<a href="https://colab.research.google.com/github/EdmilsonSantana/llm-vehicle-repair/blob/main/Assistente_do_Mecanico_TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
%pip install selenium beautifulsoup4 datasets transformers[torch]

## Scraping data from AutoZone

We are going to extract the content from the articles found in AutoZone sitemap.

In [2]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Mon_Apr__3_17:16:06_PDT_2023
Cuda compilation tools, release 12.1, V12.1.105
Build cuda_12.1.r12.1/compiler.32688072_0


In [8]:
from articles import extract_articles
import pandas as pd

In [9]:
data_dir = './data'

In [10]:
articles = extract_articles(data_dir)

In [11]:
print(articles[0])

{'title': 'Car AC Blowing Hot Air', 'content': 'Understanding the Causes\n\nA quick understanding of how air conditioning works can help with understanding what the causes could be. When AC is turned on, refrigerant that flows through the system absorbs heat from your vehicle’s cabin where it’s removed and, through a series of parts and processes, the heat is released into the atmosphere before circulating back and repeating the process. There are several points where something can be wrong, causing warm air rather than cool:\n\nThere isn’t sufficient airflow in the cabin. This could be a problem with a bad blower motor, but more commonly a plugged cabin air filter is the culprit.\nThere isn’t enough refrigerant. The gas that circulates through the system can leak out, preventing it from working efficiently.\nThe compressor may not be cycling. A clutch issue or a compressor failure can prevent the AC system from being able to disperse the heat the refrigerant has absorbed.\nThe expansi

In [12]:
df_articles = pd.DataFrame(articles)

In [13]:
df_articles.head()

,title,content,category
0,Car AC Blowing Hot Air,Understanding the Causes\n\nA quick understand...,AC & Climate Control
1,Does the Car AC Use Gas?,Introduction to Car AC and Heating Systems\n\n...,AC & Climate Control
2,How to Get Rid of a Musty Smell in Your AC,Understanding the Musty Smell\n\nOne of the fi...,AC & Climate Control
3,How to Use Car Defrosters,Types of Car Defrosters: Rear and Front Defros...,AC & Climate Control
4,Does AC Affect Gas Mileage?,How Air Conditioning Affects Fuel Consumption\...,AC & Climate Control


In [14]:
df_articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 980 entries, 0 to 979
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     980 non-null    object
 1   content   980 non-null    object
 2   category  980 non-null    object
dtypes: object(3)
memory usage: 23.1+ KB


## Bonito

In [1]:
%%capture
!git clone https://github.com/BatsResearch/bonito.git
!pip install -U bonito/

In [2]:
%%capture
!pip install --ignore-installed autoawq autoawq-kernels

In [3]:
from quantized import QuantizedBonito

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/utils/generic.py:485: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/utils/generic.py:342: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/utils/generic.py:342: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
bonito = QuantizedBonito("alexandreteles/bonito-v1-awq")

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Fusing layers...: 100%|██████████| 32/32 [00:00<00:00, 195.37it/s]


In [35]:
#df_articles.iloc[10]
# How Much Does It Cost to Replace Your A/C

title             How Much Does It Cost to Replace Your A/C? 
content     How an AC Compressor Works \n\nThe compressor ...
category                                 AC & Climate Control
Name: 10, dtype: object

In [66]:
from pprint import pprint
unannotated_paragraph = df_articles.iloc[14]['content']
pprint(unannotated_paragraph)

('How Much Does It Cost to Recharge Your Car’s AC?\n'
 'These costs are generally not that expensive when considering that this '
 'maintenance also extends the life of your compressor, keeping your air '
 'conditioning performing optimally season after season.\n'
 'A professional AC recharge cost ranges from $150 – $300 on average depending '
 'on the make and model of your vehicle. Due to\xa0refrigerant\xa0losses that '
 'occur over time, this service is one that should be added to your vehicle '
 'maintenance schedule. It is recommended that this service be performed on a '
 'vehicle every 100,000 miles or so.\n'
 'You can save on your\xa0this cost\xa0by going the DIY route and recharging '
 'your car’s AC, in which case you can expect to pay\xa0$40-60\xa0for a proper '
 'recharge kit. This\xa0may seem like\xa0a real savings over\xa0taking the car '
 'into the shop\xa0and\xa0possibly\xa0more convenient, but before '
 'proceeding\xa0with the DIY alternative, it’s important to underst

In [83]:
from transformers import set_seed
# making predictions deterministic.
#set_seed(2)

# Generate synthetic instruction tuning dataset
sampling_params = {'max_new_tokens':256, 'top_p':0.95, 'temperature':0.9, 'num_return_sequences':1}
synthetic_dataset = bonito.generate_task(
    unannotated_paragraph,
    task_type="text_gen",
    sampling_params=sampling_params
)


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Filter:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [84]:
synthetic_dataset

{'input': "What details would you include in a storyline to make it more engaging and informative?\n\nThis service is one that should be added to your vehicle maintenance schedule.\nIt's expected that at 100,000 miles, your air conditioning system will require servicing.\nRecharging your air conditioning system is not a difficult task.\nIf you overcharge the system, it is best to take the vehicle to a mechanic to have the system properly evacuated.",
 'output': '{{context}}'}

In [85]:
pprint(synthetic_dataset["input"])

('What details would you include in a storyline to make it more engaging and '
 'informative?\n'
 '\n'
 'This service is one that should be added to your vehicle maintenance '
 'schedule.\n'
 "It's expected that at 100,000 miles, your air conditioning system will "
 'require servicing.\n'
 'Recharging your air conditioning system is not a difficult task.\n'
 'If you overcharge the system, it is best to take the vehicle to a mechanic '
 'to have the system properly evacuated.')


In [86]:
pprint(synthetic_dataset["output"])

'{{context}}'


## txtinstruct

Let's follow txtinstruct tutorial from this [notebook](https://colab.research.google.com/github/neuml/txtinstruct/blob/master/examples/01_Introducing_txtinstruct.ipynb#scrollTo=7uQaAdtmH3kP).

In [6]:
%%capture
!pip install git+https://github.com/neuml/txtai git+https://github.com/neuml/txtinstruct

In [7]:
from datasets import load_dataset
from txtinstruct.models import StatementGenerator

Bulding a question generation model using the SQuAD dataset.

In [8]:
from datasets import load_dataset, Dataset
from txtinstruct.models import StatementGenerator

In [9]:
# Load SQuAD dataset
dataset = load_dataset("squad", split="train")
dataset[0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [10]:
# Train model
generator = StatementGenerator()
model, tokenizer = generator(
    "google/flan-t5-small",
    dataset,
    "sequence-sequence",
    learning_rate=1e-3,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=128 // 16,
    num_train_epochs=3,
    logging_steps=100,
)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
100,2.192000
200,2.133700
300,2.098500
400,2.068000
500,2.062900
600,2.039100
700,1.983700
800,1.863700
900,1.870700
1000,1.864100


In [24]:
#model.save_pretrained("./t5-small", from_pt=True) 

In [26]:
#tokenizer.save_pretrained('./t5-small-tokenizer')

('./t5-small-tokenizer/tokenizer_config.json',
 './t5-small-tokenizer/special_tokens_map.json',
 './t5-small-tokenizer/tokenizer.json')

In [12]:
from txtai.pipeline import Sequences

# Load statement generation model
statements = Sequences((model, tokenizer))

# Run example prompt
statements("""Generate a question using the context below.
### Context:
txtai is an open-source platform for semantic search and workflows powered by language models.""")

'What is txtai?'

Now that we have a statement generation model, let's build an instruction-tuning dataset. 

In [36]:
df_articles.rename(columns={'title': 'id', 'content': 'text'}, inplace=True)
df_articles.drop(columns=['category'], inplace=True)

In [32]:
df_articles = df_articles.replace('\n',' ', regex=True)

In [37]:
context = df_articles[:5].to_dict('records')
context

[{'id': 'Car AC Blowing Hot Air',
  'text': 'Understanding the Causes  A quick understanding of how air conditioning works can help with understanding what the causes could be. When AC is turned on, refrigerant that flows through the system absorbs heat from your vehicle’s cabin where it’s removed and, through a series of parts and processes, the heat is released into the atmosphere before circulating back and repeating the process. There are several points where something can be wrong, causing warm air rather than cool:  There isn’t sufficient airflow in the cabin. This could be a problem with a bad blower motor, but more commonly a plugged cabin air filter is the culprit. There isn’t enough refrigerant. The gas that circulates through the system can leak out, preventing it from working efficiently. The compressor may not be cycling. A clutch issue or a compressor failure can prevent the AC system from being able to disperse the heat the refrigerant has absorbed. The expansion valve i

In [38]:
# Query templates
templates = [
    "Tell me about {text}",
    "Give an explanation on {text}",
    "Provide a quick summary on {text}",
    "Explain {text} in simple terms",
    "Describe {text}"
]

In [39]:
from txtinstruct.data import DatasetBuilder

builder = DatasetBuilder(Sequences("google/flan-t5-base"), statements, templates)
builder(
    context,
    5,
    "data.json"
)

100%|██████████| 5/5 [00:00<00:00, 65331.84it/s]


In [40]:
import json

questions = json.load(open('data.json'))

In [41]:
questions[3]

{'context': 'Types of Car Defrosters: Rear and Front Defrosters  Car defrosters are found in two main places. The front is meant to defrost or defog your front windshield and the side windows, and the rear defrosters is intended to clear the back glass. Both serve the same purpose but are designed for different parts of the vehicle.  Rear defrosters are primarily responsible for clearing the rear windshield of frost, ice, and condensation. They consist of heating elements embedded in the glass, which radiate heat to melt away any obstructions. It’s triggered by hitting the defrost switch or button on the HVAC controls. Front defrosters are essential for maintaining visibility through the front windshield. They work by blowing air, usually warm air, onto the glass surface to remove fog and ice buildup. It’s activated when the heater control air direction is switched to defrost setting, whether it’s only defrost or a blend with floor or dash vents too.  How to Use Car Defrosters: Step-by